# Create 3D Models with Nerf Studio from iPad with Camera

Author: Deeepwin  
Date: 18.03.2023 
***

## Installation

This installation instructions explains in how to install Nerfstudio in a Conda enviroment that has a separte CUDA installtion. It makes the installation of Nerfstudio a bit more tricky. In my case I have installed CUDA version 11.3 in Conda which I want to use for this project. The Conda environment is called `points`. On the Ubuntu installation I run CUDA 12.1.

### Checking CUDA Installation

Make sure you have installed CUDA toolkit in Conda.

In [ ]:
!conda install -c "nvidia/label/cuda-11.3.1" cuda-toolkit
!conda install -c "nvidia/label/cuda-11.3.1" cuda-nvcc

In [ ]:
# check CUDA versions, must be 11.3.1
!conda list cudatoolkit
!conda list cudnn
!conda list cuda-nvcc

In [ ]:
# check Pytorch version, must be 1.13.1
torch_version = !pip show torch
torch_version[1]

### Setup Tiny CUDA NN

For CUDA version 11.3 you need g++ version 9 (max version is 10.2.1), see [here](https://stackoverflow.com/questions/6622454/cuda-incompatible-with-my-gcc-version). General instructions in how to setup different versions of g++ can be found [here](https://tuxamito.com/wiki/index.php/Installing_newer_GCC_versions_in_Ubuntu).

In [ ]:
!sudo apt -y install g++-9
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-9 9
!sudo update-alternatives --config g++
!g++ --version

In [ ]:
!sudo apt -y install gcc-9
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 9
!sudo update-alternatives --config gcc
!gcc --version

Now set CUDA_HOME enviroment variable. Otherwise you cannot install `tiny-cuda-nn`. Update path according to your Conda installation.

In [ ]:
!conda env config vars set CUDA_HOME="/home/martin/anaconda3/envs/points/lib/"
!conda env config vars list

Install `tiny-cuda-nn`

In [ ]:
!pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

Finally install Nerfstudio see [website](https://docs.nerf.studio/en/latest/quickstart/installation.html).

In [ ]:
!pip install nerfstudio

Pytorch automatically installs nvidia_cublas_cu11, which will cause an error in the installation of Nerfstudio. That's why we need to remove it here.

In [ ]:
!pip uninstall -y nvidia_cublas_cu11

For viszalization we use Open3D.

In [ ]:
!pip install open3d

## Train Model from Nerfstudio

In [ ]:
%%capture
# download data from nerfstudio
!ns-download-data nerfstudio --capture-name=poster

In [ ]:
# train from scratch
!ns-train nerfacto --data data/nerfstudio/poster

In [ ]:
# export as point cloud, adjust export directory
%%capture
!ns-export pointcloud --load-config outputs/poster/nerfacto/2023-03-18_122407/config.yml --output-dir exports/pcd/ --num-points 1000000 --remove-outliers True --estimate-normals False --use-bounding-box True --bounding-box-min -1 -1 -1 --bounding-box-max 1 1 1

In [ ]:
#  open in separate window
%%capture
!open3d draw exports/pcd/point_cloud.ply 

An example view of the point cloud data.

<img src="pics/chair.jpg" height=300 />


## Train your Own Model

If you have no phone with LiDAR like me, install [Kiri Engine](https://www.kiriengine.com/) app. Follow the instructions [here](https://docs.nerf.studio/en/latest/quickstart/custom_dataset.html). 

1. Once engine has processed data, export by sending link to file by email.
2. Extract file and place in `export` folder with proper name.
3. Adjust `--data` path string below. Change websocket port in case you get an error.

In [ ]:
input_dir = 'data/mine/kettle-2'

In [ ]:
!ns-train nerfacto --data $input_dir --viewer.websocket-port=7007


Open viewer and monitor progress. Once training is complete, look up output folder an adjust following variables:

In [3]:
training_id = '2023-03-18_173859'

In [4]:
import os
object_name = os.path.split(input_dir)[-1]
output_dir = 'exports/' + object_name

In [5]:
#%%capture
!ns-export pointcloud --load-config outputs/$object_name/nerfacto/$training_id/config.yml --output-dir $output_dir --num-points 1000000 --remove-outliers True --estimate-normals False --use-bounding-box True --bounding-box-min -1 -1 -1 --bounding-box-max 1 1 1

2023-03-18 17:52:52.887852: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-18 17:52:53.477701: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/martin/anaconda3/envs/points/lib/python3.7/site-packages/cv2/../../lib64:/home/martin/anaconda3/envs/finance/lib/
2023-03-18 17:52:53.477775: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mart

In [6]:
!open3d draw $output_dir/point_cloud.ply 

***************************************************
* Open3D: A Modern Library for 3D Data Processing *
*                                                 *
* Version 0.16.0                                  *
* Docs    http://www.open3d.org/docs              *
* Code    https://github.com/isl-org/Open3D       *
***************************************************
FEngine (64 bits) created at 0x7f9720c43010 (threading is enabled)
FEngine resolved backend: OpenGL
[Open3D INFO] exports/kettle-2/point_cloud.ply appears to be a point cloud
[Open3D INFO] Successfully read exports/kettle-2/point_cloud.ply


Here an example of a plant on a table.


<table><tr>
<td> <img src="pics/plant-0.jpg" height="300"/>  </td>
<td> <img src="pics/plant-1.jpg" height="300"/>  </td>
<td> <img src="pics/plant-2.jpg" height="300"/>  </td>
</tr></table>


Here an example of my Logitech vertical mouse to see the geometric accuracy.

<table><tr>
<td> <img src="pics/mouse-2-0.jpg" height="300"/>  </td>
<td> <img src="pics/mouse-2-1.jpg" height="300"/>  </td>
<td> <img src="pics/mouse-2-2.jpg" height="300"/>  </td>
<td> <img src="pics/mouse-2-3.jpg" height="300"/>  </td>
</tr></table>

Another example with reflective surfaces on a kettle.

<table><tr>
<td> <img src="pics/kettle-2-0.jpg" height="300"/>  </td>
<td> <img src="pics/kettle-2-1.jpg" height="300"/>  </td>
<td> <img src="pics/kettle-2-2.jpg" height="300"/>  </td>
<td> <img src="pics/kettle-2-3.jpg" height="300"/>  </td>
</tr></table>


## Nerfstudio Viewer

In [ ]:
!ns-viewer --load-config outputs/$object_name/nerfacto/$training_id/config.yml